<a href="https://colab.research.google.com/github/hussain0048/Natural-language-processing/blob/main/Preprocessing_Aassignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [1]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random                              # pseudo-random number generator

# **About the Twitter dataset**

# **References**

[2- Preprocessing -Assignment_Coursesteach](https://coursesteach.com/mod/page/view.php?id=6021&forceview=1)

[1-Preprocessing](https://github.com/ibrahimjelliti/Deeplearning.ai-Natural-Language-Processing-Specialization/blob/master/1%20-%20Natural%20Language%20Processing%20with%20Classification%20and%20Vector%20Spaces/Labs/Week%201/NLP_C1_W1_lecture_nb_01.ipynb)